# Interest Rates 101
<p style="color:darkblue; "><b>A TurningBull Notebook</b>

This notebook provides a foundational tutorial on interest rate concepts using Python examples. We'll explore the fundamental building blocks of fixed income analysis including discount factors, spot rates, par rates, and forward rates.

**Topics covered:**
1. Discount factors and present value calculations
2. Spot rates and zero-coupon bond pricing
3. Par rates and coupon-bearing bond pricing
4. Forward rates and expectations theory
5. Yield curves and economic interpretation
6. Yield curve bootstrapping methodology and Python implementation

Throughout this notebook, we'll use matplotlib to create visualizations that illustrate key concepts and provide hands-on Python code to demonstrate these fundamental interest rate relationships.

In [ ]:
## Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import newton
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Discount Factors

Discount factors are the foundation of all fixed income pricing. A discount factor represents the present value of $1 to be received at a future time. It answers the question: "What is $1 received in *n* years worth today?"

### Mathematical Definition

The discount factor for time *t* is given by:

$$
DF(t) = \frac{1}{(1 + r_t)^t}
$$

Where:
- $DF(t)$ is the discount factor for time $t$
- $r_t$ is the spot rate for time $t$
- $t$ is the time to maturity in years

### Key Properties
1. $DF(0) = 1$ (a dollar today is worth a dollar)
2. $DF(t) < 1$ for $t > 0$ and $r_t > 0$ (future money is worth less than present money)
3. Discount factors are monotonically decreasing with time (longer-term money is worth less)

In [ ]:
def discount_factor(rate, time):
    """
    Calculate discount factor given interest rate and time
    
    Parameters:
    rate (float): Annual interest rate (as decimal, e.g., 0.05 for 5%)
    time (float): Time to maturity in years
    
    Returns:
    float: Discount factor
    """
    return 1 / ((1 + rate) ** time)

# Example: Calculate discount factors for different time periods
rate = 0.04  # 4% annual rate
times = np.array([0.5, 1, 2, 3, 5, 7, 10])
discount_factors = [discount_factor(rate, t) for t in times]

# Create a DataFrame for better visualization
df_example = pd.DataFrame({
    'Time (Years)': times,
    'Discount Factor': discount_factors,
    'Present Value of $100': [df * 100 for df in discount_factors]
})

print("Discount Factors at 4% Annual Rate:")
print(df_example.round(4))

In [ ]:
# Visualize how discount factors change with time for different interest rates
time_range = np.linspace(0, 10, 100)
rates = [0.02, 0.04, 0.06, 0.08]

plt.figure(figsize=(12, 8))
for rate in rates:
    dfs = [discount_factor(rate, t) for t in time_range]
    plt.plot(time_range, dfs, label=f'{rate*100:.0f}% Rate', linewidth=2)

plt.xlabel('Time to Maturity (Years)')
plt.ylabel('Discount Factor')
plt.title('Discount Factors vs. Time for Different Interest Rates')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 10)
plt.ylim(0, 1)

# Add annotations
plt.annotate('Higher rates = steeper decline\nin discount factors', 
             xy=(6, 0.7), xytext=(7, 0.8),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=10, ha='center')

plt.tight_layout()
plt.show()

print("\nKey Insight: Higher interest rates lead to lower discount factors (steeper decline).")
print("This reflects the higher opportunity cost of money when rates are higher.")

## 2. Spot Rates

Spot rates (also called zero rates) are the interest rates for zero-coupon bonds of different maturities. They represent the yield on a bond that makes no intermediate payments and only pays its face value at maturity.

### Relationship with Discount Factors

Spot rates and discount factors are directly related:

$$
r_t = \left(\frac{1}{DF(t)}\right)^{1/t} - 1
$$

### Zero-Coupon Bond Pricing

The price of a zero-coupon bond is simply:

$$
P = \frac{FV}{(1 + r_t)^t} = FV \times DF(t)
$$

Where $FV$ is the face value and $P$ is the current price.

In [ ]:
def spot_rate_from_discount_factor(discount_factor, time):
    """
    Calculate spot rate from discount factor
    
    Parameters:
    discount_factor (float): Discount factor
    time (float): Time to maturity in years
    
    Returns:
    float: Spot rate (as decimal)
    """
    return (1 / discount_factor) ** (1 / time) - 1

def zero_coupon_bond_price(face_value, spot_rate, time):
    """
    Calculate zero-coupon bond price
    
    Parameters:
    face_value (float): Face value of the bond
    spot_rate (float): Spot rate (as decimal)
    time (float): Time to maturity in years
    
    Returns:
    float: Current price of the bond
    """
    return face_value / ((1 + spot_rate) ** time)

# Example: Zero-coupon bond pricing
face_value = 1000  # $1,000 face value
maturities = np.array([1, 2, 3, 5, 7, 10])
spot_rates = np.array([0.02, 0.025, 0.03, 0.035, 0.04, 0.042])  # Typical upward-sloping yield curve

bond_prices = [zero_coupon_bond_price(face_value, rate, maturity) 
               for rate, maturity in zip(spot_rates, maturities)]

zero_bond_data = pd.DataFrame({
    'Maturity (Years)': maturities,
    'Spot Rate (%)': spot_rates * 100,
    'Bond Price ($)': bond_prices,
    'Discount to Par (%)': [(face_value - price) / face_value * 100 for price in bond_prices]
})

print("Zero-Coupon Bond Pricing Example:")
print(f"Face Value: ${face_value:,}")
print(zero_bond_data.round(2))

In [ ]:
# Visualize the spot rate curve (yield curve)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left plot: Spot rate curve
ax1.plot(maturities, spot_rates * 100, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Maturity (Years)')
ax1.set_ylabel('Spot Rate (%)')
ax1.set_title('Spot Rate Curve (Yield Curve)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(1.5, 4.5)

# Add curve shape annotation
ax1.annotate('Normal upward-sloping\nyield curve', 
             xy=(5, 3.5), xytext=(7, 2.5),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=10, ha='center')

# Right plot: Zero-coupon bond prices
ax2.bar(maturities, bond_prices, alpha=0.7, color='skyblue', edgecolor='navy')
ax2.set_xlabel('Maturity (Years)')
ax2.set_ylabel('Bond Price ($)')
ax2.set_title('Zero-Coupon Bond Prices\n(Face Value: $1,000)')
ax2.grid(True, alpha=0.3)

# Add price labels on bars
for i, (maturity, price) in enumerate(zip(maturities, bond_prices)):
    ax2.text(maturity, price + 10, f'${price:.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("1. Longer maturity bonds trade at deeper discounts to par")
print("2. The upward-sloping yield curve reflects higher rates for longer maturities")
print("3. Zero-coupon bonds are always priced below par (except at maturity)")

## 3. Par Rates

Par rates are the coupon rates that would make a bond trade exactly at par (face value). These are the rates you see quoted for newly issued government bonds.

### Mathematical Definition

For a bond trading at par, the present value of all cash flows equals the face value:

$$
100 = \sum_{i=1}^{n} \frac{C}{(1 + r_i)^i} + \frac{100}{(1 + r_n)^n}
$$

Where $C$ is the annual coupon payment and $r_i$ are the spot rates.

Solving for the par rate $c$ (where $C = c \times 100$):

$$
c = \frac{1 - DF(n)}{\sum_{i=1}^{n} DF(i)}
$$

In [ ]:
def calculate_par_rate(spot_rates, maturities, target_maturity):
    """
    Calculate par rate for a given maturity using spot rates
    
    Parameters:
    spot_rates (array): Array of spot rates for each year
    maturities (array): Array of maturities corresponding to spot rates
    target_maturity (int): Maturity for which to calculate par rate
    
    Returns:
    float: Par rate (as decimal)
    """
    # Get spot rates up to target maturity
    relevant_spots = spot_rates[:target_maturity]
    relevant_maturities = maturities[:target_maturity]
    
    # Calculate discount factors
    discount_factors = [discount_factor(rate, mat) for rate, mat in zip(relevant_spots, relevant_maturities)]
    
    # Calculate par rate using formula
    sum_dfs = sum(discount_factors)
    final_df = discount_factors[-1]
    
    par_rate = (1 - final_df) / sum_dfs
    return par_rate

def price_coupon_bond(face_value, coupon_rate, spot_rates, maturities):
    """
    Price a coupon-bearing bond using spot rates
    
    Parameters:
    face_value (float): Face value of bond
    coupon_rate (float): Annual coupon rate (as decimal)
    spot_rates (array): Array of spot rates
    maturities (array): Array of maturities
    
    Returns:
    float: Bond price
    """
    annual_coupon = coupon_rate * face_value
    
    # Present value of coupon payments
    pv_coupons = sum([annual_coupon / ((1 + rate) ** mat) 
                      for rate, mat in zip(spot_rates, maturities)])
    
    # Present value of principal repayment
    pv_principal = face_value / ((1 + spot_rates[-1]) ** maturities[-1])
    
    return pv_coupons + pv_principal

# Calculate par rates for different maturities
maturities_full = np.array([1, 2, 3, 4, 5])
spot_rates_full = np.array([0.02, 0.025, 0.03, 0.035, 0.04])

par_rates = []
for i, maturity in enumerate(maturities_full, 1):
    par_rate = calculate_par_rate(spot_rates_full, maturities_full, i)
    par_rates.append(par_rate)

par_rate_data = pd.DataFrame({
    'Maturity (Years)': maturities_full,
    'Spot Rate (%)': spot_rates_full * 100,
    'Par Rate (%)': [rate * 100 for rate in par_rates]
})

print("Par Rates vs Spot Rates:")
print(par_rate_data.round(3))

In [ ]:
# Demonstrate that bonds with par rates trade at par
print("\nVerification: Bonds priced with par rates should trade at exactly $100 (par):")
print("=" * 80)

face_value = 100
for i, (maturity, par_rate) in enumerate(zip(maturities_full, par_rates)):
    # Use spot rates up to this maturity
    relevant_spots = spot_rates_full[:maturity]
    relevant_mats = maturities_full[:maturity]
    
    bond_price = price_coupon_bond(face_value, par_rate, relevant_spots, relevant_mats)
    
    print(f"{maturity}-Year Bond: Par Rate = {par_rate*100:.3f}%, Price = ${bond_price:.6f}")

# Visualize spot rates vs par rates
plt.figure(figsize=(12, 8))
plt.plot(maturities_full, spot_rates_full * 100, 'bo-', label='Spot Rates', linewidth=2, markersize=8)
plt.plot(maturities_full, [rate * 100 for rate in par_rates], 'ro-', label='Par Rates', linewidth=2, markersize=8)

plt.xlabel('Maturity (Years)')
plt.ylabel('Rate (%)')
plt.title('Spot Rates vs Par Rates')
plt.legend()
plt.grid(True, alpha=0.3)

# Add annotation about the relationship
plt.annotate('Par rates are typically\nbelow spot rates for\nupward-sloping curves', 
             xy=(3, 3.2), xytext=(4.2, 2.5),
             arrowprops=dict(arrowstyle='->', color='green'),
             fontsize=10, ha='center')

plt.tight_layout()
plt.show()

print("\nKey Insight: For upward-sloping yield curves, par rates are typically below spot rates.")
print("This is because the coupon payments in early years are discounted at lower rates.")

## 4. Forward Rates

Forward rates represent the interest rate implied by current spot rates for borrowing or lending over a future period. They are derived from the relationship between spot rates of different maturities.

### Mathematical Definition

The forward rate from time $t_1$ to time $t_2$ is:

$$
f_{t_1,t_2} = \left(\frac{(1 + r_{t_2})^{t_2}}{(1 + r_{t_1})^{t_1}}\right)^{\frac{1}{t_2-t_1}} - 1
$$

For one-year forward rates starting in year $n$:

$$
f_{n,n+1} = \frac{(1 + r_{n+1})^{n+1}}{(1 + r_n)^n} - 1
$$

### Economic Interpretation
Forward rates represent the market's expectation of future short-term interest rates, incorporating risk premiums and liquidity preferences.

In [ ]:
def calculate_forward_rate(spot_rate_1, maturity_1, spot_rate_2, maturity_2):
    """
    Calculate forward rate between two periods
    
    Parameters:
    spot_rate_1 (float): Spot rate for shorter maturity
    maturity_1 (float): Shorter maturity
    spot_rate_2 (float): Spot rate for longer maturity
    maturity_2 (float): Longer maturity
    
    Returns:
    float: Forward rate (as decimal)
    """
    return ((1 + spot_rate_2) ** maturity_2 / (1 + spot_rate_1) ** maturity_1) ** (1 / (maturity_2 - maturity_1)) - 1

def calculate_one_year_forwards(spot_rates, maturities):
    """
    Calculate one-year forward rates
    
    Parameters:
    spot_rates (array): Array of spot rates
    maturities (array): Array of maturities
    
    Returns:
    array: One-year forward rates
    """
    forwards = [spot_rates[0]]  # First forward is just the 1-year spot rate
    
    for i in range(1, len(spot_rates)):
        forward = calculate_forward_rate(spot_rates[i-1], maturities[i-1], 
                                       spot_rates[i], maturities[i])
        forwards.append(forward)
    
    return np.array(forwards)

# Calculate forward rates using our spot rate curve
forward_rates = calculate_one_year_forwards(spot_rates_full, maturities_full)

forward_data = pd.DataFrame({
    'Period': ['1Y', '1Y1Y', '1Y2Y', '1Y3Y', '1Y4Y'],
    'Description': ['Current 1-year rate', 
                   '1-year rate, 1 year forward',
                   '1-year rate, 2 years forward',
                   '1-year rate, 3 years forward',
                   '1-year rate, 4 years forward'],
    'Spot Rate (%)': spot_rates_full * 100,
    'Forward Rate (%)': forward_rates * 100
})

print("One-Year Forward Rates:")
print(forward_data.round(3))

In [ ]:
# Visualize spot rates vs one-year forward rates
plt.figure(figsize=(12, 8))

years = np.arange(1, len(spot_rates_full) + 1)
plt.plot(years, spot_rates_full * 100, 'bo-', label='Spot Rates', linewidth=2, markersize=8)
plt.plot(years, forward_rates * 100, 'ro-', label='1-Year Forward Rates', linewidth=2, markersize=8)

plt.xlabel('Year')
plt.ylabel('Rate (%)')
plt.title('Spot Rates vs One-Year Forward Rates')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(years)

# Add annotations
plt.annotate('Forward rates rise more\nsteeply than spot rates', 
             xy=(4, 5.1), xytext=(3.2, 4.5),
             arrowprops=dict(arrowstyle='->', color='green'),
             fontsize=10, ha='center')

plt.tight_layout()
plt.show()

# Demonstrate forward rate relationship
print("\nForward Rate Verification:")
print("If you invest for 2 years at the 2-year spot rate vs.")
print("investing for 1 year then rolling over at the 1Y1Y forward rate:")
print("\nStrategy 1 (2-year spot):")
strategy1_return = (1 + spot_rates_full[1]) ** 2
print(f"$1 invested at {spot_rates_full[1]*100:.1f}% for 2 years = ${strategy1_return:.6f}")

print("\nStrategy 2 (1-year + 1Y1Y forward):")
strategy2_return = (1 + spot_rates_full[0]) * (1 + forward_rates[1])
print(f"$1 at {spot_rates_full[0]*100:.1f}% for 1 year, then at {forward_rates[1]*100:.2f}% for 1 year = ${strategy2_return:.6f}")

print(f"\nDifference: ${abs(strategy1_return - strategy2_return):.10f} (should be approximately zero)")

## 5. Yield Curves and Economic Interpretation

A yield curve is a graphical representation of interest rates across different maturities. The shape of the yield curve provides valuable insights into economic conditions and market expectations.

### Common Yield Curve Shapes

1. **Normal (Upward-sloping)**: Long-term rates higher than short-term rates
   - Indicates economic growth expectations
   - Reflects term premium for longer-term lending

2. **Inverted (Downward-sloping)**: Long-term rates lower than short-term rates
   - Often predicts economic recession
   - Suggests expectations of falling future rates

3. **Flat**: Similar rates across all maturities
   - Indicates economic uncertainty
   - Transition between normal and inverted

4. **Humped**: Rates peak at intermediate maturities
   - Less common shape
   - Often occurs during monetary policy transitions

In [ ]:
# Create examples of different yield curve shapes
maturities_curve = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])

# Normal yield curve (current economic expansion)
normal_curve = np.array([1.5, 1.8, 2.1, 2.8, 3.2, 3.6, 3.9, 4.1, 4.3, 4.4])

# Inverted yield curve (recession expectations)
inverted_curve = np.array([4.5, 4.2, 3.8, 3.2, 2.8, 2.4, 2.2, 2.1, 2.0, 1.9])

# Flat yield curve (economic uncertainty)
flat_curve = np.array([3.1, 3.0, 3.0, 3.1, 3.0, 2.9, 3.0, 3.1, 3.0, 2.9])

# Humped yield curve (monetary policy transition)
humped_curve = np.array([2.0, 2.5, 3.2, 4.1, 4.3, 4.2, 3.9, 3.6, 3.4, 3.3])

# Create visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Normal curve
ax1.plot(maturities_curve, normal_curve, 'b-o', linewidth=2, markersize=6)
ax1.set_title('Normal Yield Curve\n(Economic Growth Expected)', fontweight='bold')
ax1.set_ylabel('Yield (%)')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 5)

# Inverted curve
ax2.plot(maturities_curve, inverted_curve, 'r-o', linewidth=2, markersize=6)
ax2.set_title('Inverted Yield Curve\n(Recession Expected)', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 5)

# Flat curve
ax3.plot(maturities_curve, flat_curve, 'g-o', linewidth=2, markersize=6)
ax3.set_title('Flat Yield Curve\n(Economic Uncertainty)', fontweight='bold')
ax3.set_xlabel('Maturity (Years)')
ax3.set_ylabel('Yield (%)')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 5)

# Humped curve
ax4.plot(maturities_curve, humped_curve, 'm-o', linewidth=2, markersize=6)
ax4.set_title('Humped Yield Curve\n(Policy Transition)', fontweight='bold')
ax4.set_xlabel('Maturity (Years)')
ax4.grid(True, alpha=0.3)
ax4.set_ylim(0, 5)

plt.tight_layout()
plt.show()

# Calculate and display curve statistics
curves_data = {
    'Normal': normal_curve,
    'Inverted': inverted_curve,
    'Flat': flat_curve,
    'Humped': humped_curve
}

print("Yield Curve Statistics:")
print("=" * 60)
for name, curve in curves_data.items():
    slope_2_10 = curve[7] - curve[2]  # 10Y - 1Y spread
    steepness = curve[-1] - curve[0]  # 30Y - 3M spread
    print(f"{name:>8}: 10Y-1Y Spread = {slope_2_10:+5.1f}bp, 30Y-3M Spread = {steepness:+5.1f}bp")

### Yield Curve Risk Factors

Yield curves can be decomposed into key risk factors that drive their movements:

1. **Level**: Parallel shifts up or down across all maturities
2. **Slope**: Steepening or flattening (long rates vs short rates)
3. **Curvature**: Changes in the curve's bend (butterfly movements)

In [ ]:
# Demonstrate yield curve risk factors
base_curve = normal_curve.copy()

# Level shift: +100bp parallel shift
level_shift = base_curve + 1.0

# Slope change: steepening (short rates -50bp, long rates +50bp)
slope_change = base_curve.copy()
steepening_factor = np.linspace(-0.5, 0.5, len(maturities_curve))
slope_change += steepening_factor

# Curvature change: butterfly (short and long rates +25bp, medium rates -50bp)
curvature_change = base_curve.copy()
# Create a butterfly pattern
butterfly_factor = -2 * np.exp(-((maturities_curve - 5) ** 2) / 8) + 0.5
curvature_change += butterfly_factor * 0.5

# Visualize the risk factors
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Level shift
ax1.plot(maturities_curve, base_curve, 'b-o', label='Original', linewidth=2)
ax1.plot(maturities_curve, level_shift, 'r--s', label='+100bp Level', linewidth=2)
ax1.set_title('Level Risk Factor\n(Parallel Shift)')
ax1.set_xlabel('Maturity (Years)')
ax1.set_ylabel('Yield (%)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Slope change
ax2.plot(maturities_curve, base_curve, 'b-o', label='Original', linewidth=2)
ax2.plot(maturities_curve, slope_change, 'g--s', label='Steepening', linewidth=2)
ax2.set_title('Slope Risk Factor\n(Steepening/Flattening)')
ax2.set_xlabel('Maturity (Years)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Curvature change
ax3.plot(maturities_curve, base_curve, 'b-o', label='Original', linewidth=2)
ax3.plot(maturities_curve, curvature_change, 'm--s', label='Butterfly', linewidth=2)
ax3.set_title('Curvature Risk Factor\n(Butterfly Movement)')
ax3.set_xlabel('Maturity (Years)')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Yield Curve Risk Factors:")
print("1. LEVEL: Affects all maturities equally (monetary policy, inflation expectations)")
print("2. SLOPE: Affects short vs long rates differently (economic growth, term premium)")
print("3. CURVATURE: Affects the bend in the curve (supply/demand, convexity hedging)")

## 6. Yield Curve Bootstrapping

Bootstrapping is the process of constructing a zero-coupon yield curve from the prices of coupon-bearing bonds. This is essential because zero-coupon bonds may not be available for all maturities, but we need the spot rate curve for accurate pricing.

### The Bootstrapping Process

1. Start with the shortest maturity (usually a Treasury bill or money market rate)
2. Use the next shortest maturity bond to solve for the next spot rate
3. Continue iteratively, using previously calculated spot rates to solve for longer rates
4. For each bond, the sum of discounted cash flows must equal the bond price

### Mathematical Approach

For a 2-year bond with annual coupons:
$$
P = \frac{C}{1 + r_1} + \frac{C + FV}{(1 + r_2)^2}
$$

Given $P$, $C$, $FV$, and $r_1$, we can solve for $r_2$.

In [ ]:
def bootstrap_yield_curve(bond_data):
    """
    Bootstrap a zero-coupon yield curve from coupon bond data
    
    Parameters:
    bond_data (DataFrame): DataFrame with columns 'maturity', 'coupon_rate', 'price'
    
    Returns:
    DataFrame: Bootstrapped spot rates
    """
    # Sort by maturity
    bond_data = bond_data.sort_values('maturity')
    
    spot_rates = []
    face_value = 100  # Assume par value of 100
    
    for i, row in bond_data.iterrows():
        maturity = row['maturity']
        coupon_rate = row['coupon_rate']
        price = row['price']
        annual_coupon = coupon_rate * face_value
        
        if maturity == 1:
            # For 1-year bond, solve directly
            # price = (coupon + face_value) / (1 + r1)
            spot_rate = (annual_coupon + face_value) / price - 1
        else:
            # For multi-year bonds, use previously calculated spot rates
            # Define function to solve for spot rate
            def bond_price_equation(r_n):
                pv = 0
                # Present value of intermediate coupon payments
                for year in range(1, int(maturity)):
                    pv += annual_coupon / ((1 + spot_rates[year-1]) ** year)
                # Present value of final payment
                pv += (annual_coupon + face_value) / ((1 + r_n) ** maturity)
                return pv - price
            
            # Solve for the spot rate
            spot_rate = newton(bond_price_equation, 0.05)  # Initial guess of 5%
        
        spot_rates.append(spot_rate)
    
    return pd.DataFrame({
        'maturity': bond_data['maturity'].values,
        'spot_rate': spot_rates
    })

# Create sample bond data for bootstrapping
sample_bonds = pd.DataFrame({
    'maturity': [1, 2, 3, 4, 5],
    'coupon_rate': [0.02, 0.025, 0.03, 0.035, 0.04],  # Annual coupon rates
    'price': [101.5, 102.2, 103.1, 104.8, 106.2]      # Market prices
})

print("Input Bond Data for Bootstrapping:")
print(sample_bonds.round(3))

# Bootstrap the yield curve
bootstrapped_curve = bootstrap_yield_curve(sample_bonds)

print("\nBootstrapped Spot Rates:")
print(bootstrapped_curve.round(4))

In [ ]:
# Verify the bootstrapping by pricing the original bonds using bootstrapped rates
def verify_bootstrap(bond_data, spot_rates_df):
    """
    Verify bootstrapping by repricing bonds using calculated spot rates
    """
    verification_results = []
    face_value = 100
    
    for i, bond in bond_data.iterrows():
        maturity = bond['maturity']
        coupon_rate = bond['coupon_rate']
        market_price = bond['price']
        annual_coupon = coupon_rate * face_value
        
        # Calculate theoretical price using spot rates
        theoretical_price = 0
        for year in range(1, int(maturity) + 1):
            spot_rate = spot_rates_df[spot_rates_df['maturity'] == year]['spot_rate'].iloc[0]
            if year == maturity:
                cash_flow = annual_coupon + face_value
            else:
                cash_flow = annual_coupon
            theoretical_price += cash_flow / ((1 + spot_rate) ** year)
        
        verification_results.append({
            'maturity': maturity,
            'coupon_rate': coupon_rate * 100,
            'market_price': market_price,
            'theoretical_price': theoretical_price,
            'difference': theoretical_price - market_price
        })
    
    return pd.DataFrame(verification_results)

# Verify the bootstrapping
verification = verify_bootstrap(sample_bonds, bootstrapped_curve)

print("Bootstrapping Verification:")
print("(Theoretical prices should match market prices)")
print("=" * 70)
print(verification.round(6))

# Visualize the bootstrapped curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Left plot: Bootstrapped spot rates
ax1.plot(bootstrapped_curve['maturity'], bootstrapped_curve['spot_rate'] * 100, 
         'bo-', linewidth=2, markersize=8, label='Bootstrapped Spot Rates')
ax1.plot(sample_bonds['maturity'], sample_bonds['coupon_rate'] * 100, 
         'rs--', linewidth=2, markersize=8, label='Bond Coupon Rates')
ax1.set_xlabel('Maturity (Years)')
ax1.set_ylabel('Rate (%)')
ax1.set_title('Bootstrapped Zero-Coupon Yield Curve')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right plot: Price verification
ax2.plot(verification['maturity'], verification['market_price'], 
         'bo-', linewidth=2, markersize=8, label='Market Prices')
ax2.plot(verification['maturity'], verification['theoretical_price'], 
         'rx--', linewidth=2, markersize=8, label='Theoretical Prices')
ax2.set_xlabel('Maturity (Years)')
ax2.set_ylabel('Bond Price ($)')
ax2.set_title('Price Verification\n(Market vs Theoretical)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

max_error = verification['difference'].abs().max()
print(f"\nMaximum pricing error: ${max_error:.8f}")
print("(Should be very close to zero for successful bootstrap)")

### Advanced Bootstrapping Considerations

In practice, bootstrapping involves several additional complexities:

1. **Day Count Conventions**: Different markets use different day counting methods
2. **Interpolation**: Missing maturities require interpolation techniques
3. **Settlement Dates**: Bonds settle T+1 or T+2, affecting calculations
4. **Accrued Interest**: Must account for interest accrued since last coupon payment
5. **Smoothing**: Real market data may require curve smoothing techniques

In [ ]:
# Demonstrate interpolation for missing maturities
from scipy.interpolate import interp1d

def interpolate_yield_curve(known_maturities, known_rates, target_maturities, method='cubic'):
    """
    Interpolate yield curve for missing maturities
    
    Parameters:
    known_maturities (array): Maturities with known rates
    known_rates (array): Known spot rates
    target_maturities (array): Maturities needing interpolation
    method (str): Interpolation method ('linear', 'cubic', etc.)
    
    Returns:
    array: Interpolated rates
    """
    interpolator = interp1d(known_maturities, known_rates, kind=method, 
                           fill_value='extrapolate')
    return interpolator(target_maturities)

# Create a more realistic scenario with missing maturities
known_maturities = np.array([0.25, 0.5, 1, 2, 5, 10, 30])
known_rates = np.array([1.5, 1.8, 2.1, 2.8, 3.6, 4.1, 4.4]) / 100

# Target maturities (including missing ones)
all_maturities = np.array([0.25, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 7, 10, 15, 20, 30])

# Interpolate missing rates
interpolated_rates = interpolate_yield_curve(known_maturities, known_rates, all_maturities)

# Visualize interpolation
plt.figure(figsize=(12, 8))
plt.plot(known_maturities, known_rates * 100, 'ro', markersize=10, label='Known Rates', zorder=3)
plt.plot(all_maturities, interpolated_rates * 100, 'b-', linewidth=2, label='Interpolated Curve')
plt.plot(all_maturities, interpolated_rates * 100, 'b.', markersize=6, alpha=0.7)

# Highlight interpolated points
missing_mask = ~np.isin(all_maturities, known_maturities)
plt.plot(all_maturities[missing_mask], interpolated_rates[missing_mask] * 100, 
         'gs', markersize=8, label='Interpolated Points', zorder=2)

plt.xlabel('Maturity (Years)')
plt.ylabel('Spot Rate (%)')
plt.title('Yield Curve Interpolation\n(Cubic Spline Method)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 32)

plt.tight_layout()
plt.show()

# Display interpolation results
interpolation_results = pd.DataFrame({
    'Maturity': all_maturities,
    'Rate (%)': interpolated_rates * 100,
    'Type': ['Known' if mat in known_maturities else 'Interpolated' 
             for mat in all_maturities]
})

print("Yield Curve Interpolation Results:")
print(interpolation_results.round(3))

## Summary

This notebook has covered the essential concepts of interest rate analysis and yield curve construction:

### Key Takeaways:

1. **Discount Factors**: The foundation of all fixed income pricing, representing the present value of future cash flows. They decrease monotonically with time and increase in sensitivity to rate changes for longer maturities.

2. **Spot Rates**: Zero-coupon rates that form the basis for pricing all fixed income securities. They have a direct mathematical relationship with discount factors and represent the "pure" interest rate for each maturity.

3. **Par Rates**: The coupon rates that would make bonds trade at par value. For upward-sloping yield curves, par rates are typically below corresponding spot rates due to the present value effect of early coupon payments.

4. **Forward Rates**: Implied future interest rates derived from current spot rates. They represent market expectations of future short-term rates and are essential for understanding yield curve dynamics.

5. **Yield Curve Shapes**: Different curve shapes provide valuable economic insights:
   - Normal curves suggest economic growth
   - Inverted curves often predict recessions
   - Flat curves indicate uncertainty
   - Risk factors (level, slope, curvature) drive curve movements

6. **Bootstrapping**: A critical technique for constructing zero-coupon yield curves from coupon bond prices. The iterative process allows us to extract spot rates when zero-coupon bonds aren't available for all maturities.

### Practical Applications:

- **Bond Pricing**: Use spot rates to accurately price any fixed income security
- **Risk Management**: Understand duration and convexity through discount factor sensitivity
- **Economic Analysis**: Interpret yield curve shapes for macroeconomic insights
- **Portfolio Management**: Apply forward rates for horizon analysis and total return forecasting
- **Derivatives Pricing**: Foundation for pricing interest rate swaps, options, and other derivatives

These fundamental concepts provide the building blocks for more advanced fixed income analysis, including duration, convexity, and complex derivative valuation. Understanding the relationships between discount factors, spot rates, par rates, and forward rates is essential for anyone working in fixed income markets, risk management, or quantitative finance.